### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [15]:
num_iterations = 1
empty_list = []
for i in range(500, 600):
    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.96


######### Lightcurve features #########

amplitude = 0.02826528605631
cusum = 0.06097634599287094
eta = 2.568956109517662
hl_amp_ratio = 0.7904217398319595
kurtosis = 0.6398422296514017
n_points = 444
period = 0.9153807184455794
period_SNR = 21.522342810535818
period_log10FAP = -4.61131176051055
period_uncertainty = 0.00034407639549927094
phase_cusum = 0.1334517262101184
phase_eta = 2.2662047258468343
phi21 = -0.37418098737448535
phi31 = 0.10943924283230722
quartile31 = 0.08664610411643459
r21 = 0.24107699660712154
r31 = 0.23255005413597227
shapiro_w = 0.9882878065109253
skewness = -0.14449512286804508
slope_per10 = -0.18930877887233039
slope_per90 = 0.2186778463060268
stetson_k = 0.7988012330263118
weighted_mean = 15.573004607930406
weighted_std = 0.05426843693102455
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4466782406855245
cusum = 0.18796148134229448
eta = 1.5260267197624837
hl_amp_ratio = 0.2518409805080718
kurtosis = -0.36369903250166
n_points = 992
period = 0.5059121460121734
period_SNR = 194.4855547397528
period_log10FAP = -138.78712278734153
period_uncertainty = 0.00011004470053987969
phase_cusum = 0.3423654792095024
phase_eta = 0.17789953566215638
phi21 = 3.7226175856926336
phi31 = 4.549565969512931
quartile31 = 0.5769482196924773
r21 = 0.4175833378723108
r31 = 0.32660912979073586
shapiro_w = 0.8606741428375244
skewness = -0.9343618905059771
slope_per10 = -0.04185076896977931
slope_per90 = 0.03574197192600539
stetson_k = 0.6594266628291016
weighted_mean = 14.80056600177701
weighted_std = 0.3943595876874736
Pulled 1 of 1


...



######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.1770257311871845
cusum = 0.06088795245692066
eta = 2.0528420574905546
hl_amp_ratio = 0.3928907790240969
kurtosis = -0.09287247165026047
n_points = 560
period = 0.4745411067550701
period_SNR = 130.98303371938772
period_log10FAP = -69.34640414588942
period_uncertainty = 9.78300838902213e-05
phase_cusum = 0.23202229168455504
phase_eta = 0.4032925908014643
phi21 = -2.4257373927467905
phi31 = -1.5934108862127008
quartile31 = 0.2186020441298524
r21 = 0.48122058873629947
r31 = 0.26816932835420787
shapiro_w = 0.9130770564079285
skewness = -0.8920414970794436
slope_per10 = -0.1283041712883579
slope_per90 = 0.09433293034723622
stetson_k = 0.7443198343213596
weighted_mean = 14.52134727424145
weighted_std = 0.1460374116900185
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.7


######### Lightcurve features #########

amplitude = 0.23073264056178078
cusum = 0.13229711877162378
eta = 2.3715774875880595
hl_amp_ratio = 0.802716466646756
kurtosis = -0.3131813325422059
n_points = 879
period = 0.28959714786776386
period_SNR = 161.08043010179543
period_log10FAP = -106.08775229159367
period_uncertainty = 3.006115583617519e-05
phase_cusum = 0.25623708620427876
phase_eta = 0.9081521302718295
phi21 = 1.0892859401324002
phi31 = -0.7125628531580218
quartile31 = 0.30267682746671554
r21 = 0.11354205948771985
r31 = 0.04880615790810383
shapiro_w = 0.9906668066978455
skewness = -0.19244210493407937
slope_per10 = -0.01282497716214764
slope_per90 = 0.013092642725666176
stetson_k = 0.7901794687829093
weighted_mean = 15.616361905286292
weighted_std = 0.19900362167132968
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.45


######### Lightcurve features #########

amplitude = 0.10486684379524373
cusum = 0.18926525430399083
eta = 6.125505125891312
hl_amp_ratio = 1.2907921009233356
kurtosis = 70.16420868479423
n_points = 958
period = 0.20491771819637913
period_SNR = 49.981932456068016
period_log10FAP = -20.1792504804736
period_uncertainty = 2.432329992980986e-05
phase_cusum = 0.33757496238544404
phase_eta = 5.460852514731833
phi21 = -1.3856255130609367
phi31 = -4.965141017423245
quartile31 = 0.18793408668822487
r21 = 0.27019591074820787
r31 = 0.15975696395116654
shapiro_w = 0.45594799518585205
skewness = 7.379721979995442
slope_per10 = -0.023038833536442482
slope_per90 = 0.02273448155585883
stetson_k = 0.868433352969031
weighted_mean = 13.21114136384718
weighted_std = 0.11754777912177648
